In [9]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.nn.init as weight_init
import numpy as np
from time import time
import data
import sys
from metrics import Metrics
import random
import matplotlib.pyplot as plt
%matplotlib inline
DEVICE = torch.device('cuda:0')


In [16]:
def config_HRED():
    conf = {
    'maxlen':30, # maximum utterance length
    'diaglen':10, # how many utterance kept in the context window

# Model Arguments
    'emb_size':200, # size of word embeddings
    'n_hidden':300, # number of hidden units per layer
    'n_layers':1, # number of layers
    'noise_radius':0.2, # stdev of noise for autoencoder (regularizer)
    'z_size':200, # dimension of z # 300 performs worse
    'lambda_gp':10, # Gradient penalty lambda hyperparameter.
    'temp':1.0, # softmax temperature (lower --> more discrete)
    'dropout':0.5, # dropout applied to layers (0 = no dropout)

# Training Arguments
    'batch_size':32,
    'epochs':100, # maximum number of epochs
    'min_epochs':2, # minimum number of epochs to train for

    'n_iters_d':5, # number of discriminator iterations in training
    'lr_ae':1.0, # autoencoder learning rate
    'lr_gan_g':5e-05, # generator learning rate
    'lr_gan_d':1e-05, # critic/discriminator learning rate
    'beta1':0.9, # beta1 for adam
    'clip':1.0,  # gradient clipping, max norm
    'gan_clamp':0.01,  # WGAN clamp (Do not use clamp when you apply gradient penelty             
    }
    return conf 

config = config_HRED()
random.seed(1111)

In [11]:
def gData(data):
    tensor=data
    if isinstance(data, np.ndarray):
        tensor = torch.from_numpy(data)
    tensor=tensor.to(DEVICE)
    return tensor
def gVar(data):
    return gData(data)
def print_flush(data, args=None):
    if args == None:
        print(data)
    else:
        print(data, args)
    sys.stdout.flush()
    
def indexes2sent(indexes, vocab, eos_tok, ignore_tok=0): 
    '''indexes: numpy array'''
    def revert_sent(indexes, ivocab, eos_tok, ignore_tok=0):
        toks=[]
        length=0
        indexes=filter(lambda i: i!=ignore_tok, indexes)
        for idx in indexes:
            toks.append(ivocab[idx])
            length+=1
            if idx == eos_tok:
                break
        return ' '.join(toks), length
    
    ivocab = {v: k for k, v in vocab.items()}
    if indexes.ndim==1:# one sentence
        return revert_sent(indexes, ivocab, eos_tok, ignore_tok)
    else:# dim>1
        sentences=[] # a batch of sentences
        lens=[]
        for inds in indexes:
            sentence, length = revert_sent(inds, ivocab, eos_tok, ignore_tok)
            sentences.append(sentence)
            lens.append(length)
        return sentences, lens

In [23]:
class Encoder(nn.Module):
    def __init__(self, embedder, input_size, hidden_size, bidirectional, n_layers, noise_radius=0.2):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.noise_radius=noise_radius
        self.n_layers = n_layers
        self.bidirectional = bidirectional
        assert type(self.bidirectional)==bool
        self.embedding = embedder
        self.rnn = nn.GRU(input_size, hidden_size, n_layers, batch_first=True, bidirectional=bidirectional)
        self.init_weights()
        
    def init_weights(self):
        for w in self.rnn.parameters(): 
            if w.dim()>1:
                weight_init.orthogonal_(w)
                
    def store_grad_norm(self, grad):
        norm = torch.norm(grad, 2, 1)
        self.grad_norm = norm.detach().data.mean()
        return grad
    
    def forward(self, inputs, input_lens=None, noise=False): 
        if self.embedding is not None:
            inputs=self.embedding(inputs) 
        
        batch_size, seq_len, emb_size=inputs.size()
#         inputs=F.dropout(inputs, 0.5, self.training)
        
        if input_lens is not None:
            input_lens_sorted, indices = input_lens.sort(descending=True)
            inputs_sorted = inputs.index_select(0, indices)        
            inputs = pack_padded_sequence(inputs_sorted, input_lens_sorted.data.tolist(), batch_first=True)
            
        init_hidden = gVar(torch.zeros(self.n_layers*(1+self.bidirectional), batch_size, self.hidden_size))
        hids, h_n = self.rnn(inputs, init_hidden) 
        if input_lens is not None:
            _, inv_indices = indices.sort()
            hids, lens = pad_packed_sequence(hids, batch_first=True)     
            hids = hids.index_select(0, inv_indices)
            h_n = h_n.index_select(1, inv_indices)
        h_n = h_n.view(self.n_layers, (1+self.bidirectional), batch_size, self.hidden_size)
        h_n = h_n[-1]
        enc = h_n.transpose(1,0).contiguous().view(batch_size,-1) 
#         if noise and self.noise_radius > 0:
#             gauss_noise = gVar(torch.normal(means=torch.zeros(enc.size()),std=self.noise_radius))
#             enc = enc + gauss_noise
            
        return enc, hids
    
class ContextEncoder(nn.Module):
    def __init__(self, utt_encoder, input_size, hidden_size, n_layers=1, noise_radius=0.2):
        super(ContextEncoder, self).__init__()
        self.hidden_size = hidden_size
        self.noise_radius=noise_radius
        
        self.n_layers = n_layers
        
        self.utt_encoder=utt_encoder
        self.rnn = nn.GRU(input_size, hidden_size, batch_first=True)
        self.init_weights()
        
    def init_weights(self):
        for w in self.rnn.parameters(): # initialize the gate weights with orthogonal
            if w.dim()>1:
                weight_init.orthogonal_(w)
    
    def store_grad_norm(self, grad):
        norm = torch.norm(grad, 2, 1)
        self.grad_norm = norm.detach().data.mean()
        return grad

    def forward(self, context, context_lens, utt_lens, floors, noise=False): 
        batch_size, max_context_len, max_utt_len = context.size()
        utts=context.view(-1, max_utt_len) 
        utt_lens=utt_lens.view(-1)
        utt_encs,_ = self.utt_encoder(utts, utt_lens) 
        utt_encs = utt_encs.view(batch_size, max_context_len, -1)
        floor_one_hot = gVar(torch.zeros(floors.numel(), 2))
        floor_one_hot.data.scatter_(1, floors.view(-1, 1), 1)
        floor_one_hot = floor_one_hot.view(-1, max_context_len, 2)
        utt_floor_encs = torch.cat([utt_encs, floor_one_hot], 2) 
        
#         utt_floor_encs=F.dropout(utt_floor_encs, 0.25, self.training)
        context_lens_sorted, indices = context_lens.sort(descending=True)
        utt_floor_encs = utt_floor_encs.index_select(0, indices)
        utt_floor_encs = pack_padded_sequence(utt_floor_encs, context_lens_sorted.data.tolist(), batch_first=True)
        
        init_hidden=gVar(torch.zeros(1, batch_size, self.hidden_size))
        hids, h_n = self.rnn(utt_floor_encs, init_hidden)
        
        _, inv_indices = indices.sort()
        h_n = h_n.index_select(1, inv_indices)  
        enc = h_n.transpose(1,0).contiguous().view(batch_size, -1)

#         if noise and self.noise_radius > 0:
#             gauss_noise = gVar(torch.normal(means=torch.zeros(enc.size()),std=self.noise_radius))
#             enc = enc + gauss_noise
        return enc
class Variation(nn.Module):
    def __init__(self, input_size, z_size):
        super(Variation, self).__init__()
        self.input_size = input_size
        self.z_size=z_size   
        self.fc = nn.Sequential(
            nn.Linear(input_size, z_size),
            nn.BatchNorm1d(z_size, eps=1e-05, momentum=0.1),
            nn.Tanh(),
            nn.Linear(z_size, z_size),
            nn.BatchNorm1d(z_size, eps=1e-05, momentum=0.1),
            nn.Tanh(),
        )
        self.context_to_mu=nn.Linear(z_size, z_size) # activation???
        self.context_to_logsigma=nn.Linear(z_size, z_size) 
        
        self.fc.apply(self.init_weights)
        self.init_weights(self.context_to_mu)
        self.init_weights(self.context_to_logsigma)
        
    def init_weights(self, m):
        if isinstance(m, nn.Linear):        
            m.weight.data.uniform_(-0.02, 0.02)
            m.bias.data.fill_(0)

    def forward(self, context):
        batch_size,_=context.size()
        context = self.fc(context)
        mu=self.context_to_mu(context)
        logsigma = self.context_to_logsigma(context) 
        std = torch.exp(0.5 * logsigma)
        
        epsilon = gVar(torch.randn([batch_size, self.z_size]))
        z = epsilon * std + mu  
        return z, mu, logsigma 
    
class Decoder(nn.Module):
    def __init__(self, embedder, input_size, hidden_size, vocab_size, n_layers=1):
        super(Decoder, self).__init__()
        self.n_layers = n_layers
        self.input_size= input_size 
        self.hidden_size = hidden_size 
        self.vocab_size = vocab_size 
        self.embedding = embedder
#         self.linear = nn.Linear(500, hidden_size)
        self.rnn = nn.GRU(input_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, vocab_size)
        self.init_weights()
        
    def init_weights(self):
        initrange = 0.1
        for w in self.rnn.parameters():
            if w.dim()>1:
                weight_init.orthogonal_(w)
        self.out.weight.data.uniform_(-initrange, initrange)
        self.out.bias.data.fill_(0)
    
    def forward(self, init_hidden, context=None, inputs=None, lens=None):
        batch_size, maxlen = inputs.size()
        if self.embedding is not None:
            inputs = self.embedding(inputs)
        if context is not None:
            repeated_context = context.unsqueeze(1).repeat(1, maxlen, 1)
            inputs = torch.cat([inputs, repeated_context], 2)
#         inputs = F.dropout(inputs, 0.5, self.training)  ß
#         init_hidden = self.linear(init_hidden)
        hids, h_n = self.rnn(inputs, init_hidden.unsqueeze(0))
        decoded = self.out(hids.contiguous().view(-1, self.hidden_size))# reshape before linear over vocab
        decoded = decoded.view(batch_size, maxlen, self.vocab_size)
        return decoded
    
    def sampling(self, init_hidden, context, maxlen, SOS_tok, EOS_tok, mode='greedy'):
        batch_size=init_hidden.size(0)
#         init_hidden = self.linear(init_hidden)
        decoded_words = np.zeros((batch_size, maxlen), dtype=np.int)
        sample_lens=np.zeros(batch_size, dtype=np.int)         
        decoder_input = gVar(torch.LongTensor([[SOS_tok]*batch_size]).view(batch_size,1))
        decoder_input = self.embedding(decoder_input) if self.embedding is not None else decoder_input 
        decoder_input = torch.cat([decoder_input, context.unsqueeze(1)],2) if context is not None else decoder_input
        decoder_hidden = init_hidden.unsqueeze(0).contiguous()
        for di in range(maxlen):
            decoder_output, decoder_hidden = self.rnn(decoder_input, decoder_hidden)
            decoder_output=self.out(decoder_output)
            if mode=='greedy':
                topi = decoder_output[:,-1].max(1, keepdim=True)[1] 
            elif mode=='sample':
                topi = torch.multinomial(F.softmax(decoder_output[:,-1], dim=1), 1)                    
            decoder_input = self.embedding(topi) if self.embedding is not None else topi
            decoder_input = torch.cat([decoder_input, context.unsqueeze(1)],2) if context is not None else decoder_input
            ni = topi.squeeze().data.cpu().numpy() 
            decoded_words[:,di]=ni
                      
        for i in range(batch_size):
            for word in decoded_words[i]:
                if word == EOS_tok:
                    break
                sample_lens[i]=sample_lens[i]+1
        return decoded_words, sample_lens

In [24]:
one = gData(torch.FloatTensor([1]))
minus_one = one * -1    
class DialogWAE(nn.Module):
    def __init__(self, config, vocab_size, PAD_token=0):
        super(DialogWAE, self).__init__()
        self.vocab_size = vocab_size
        self.maxlen=config['maxlen']
        self.clip = config['clip']
        self.lambda_gp = config['lambda_gp']
        self.temp=config['temp']
        
        self.embedder= nn.Embedding(vocab_size, config['emb_size'], padding_idx=PAD_token)
        self.utt_encoder = Encoder(self.embedder, config['emb_size'], config['n_hidden'], 
                                   True, config['n_layers'], config['noise_radius']) 
        self.context_encoder = ContextEncoder(self.utt_encoder, config['n_hidden']*2+2, config['n_hidden'], 1, config['noise_radius']) 
        self.prior_net = Variation(config['n_hidden'], config['z_size']) # p(e|c)
        self.post_net = Variation(config['n_hidden']*3, config['z_size']) # q(e|c,x)
        
        self.post_generator = nn.Sequential( 
            nn.Linear(config['z_size'], config['z_size']),
            nn.BatchNorm1d(config['z_size'], eps=1e-05, momentum=0.1),
            nn.ReLU(),
            nn.Linear(config['z_size'], config['z_size']),
            nn.BatchNorm1d(config['z_size'], eps=1e-05, momentum=0.1),
            nn.ReLU(),
            nn.Linear(config['z_size'], config['z_size'])
        )
        self.post_generator.apply(self.init_weights)
                                                                              
        self.prior_generator = nn.Sequential( 
            nn.Linear(config['z_size'], config['z_size']),
            nn.BatchNorm1d(config['z_size'], eps=1e-05, momentum=0.1),
            nn.ReLU(),
            nn.Linear(config['z_size'], config['z_size']),
            nn.BatchNorm1d(config['z_size'], eps=1e-05, momentum=0.1),
            nn.ReLU(),
            nn.Linear(config['z_size'], config['z_size'])
        ) 
        self.prior_generator.apply(self.init_weights)
                                                                                             
        self.decoder = Decoder(self.embedder, config['emb_size'], config['n_hidden']+config['z_size'], 
                               vocab_size, n_layers=1) 
        
        self.discriminator = nn.Sequential(  
            nn.Linear(config['n_hidden']+config['z_size'], config['n_hidden']*2),
            nn.BatchNorm1d(config['n_hidden']*2, eps=1e-05, momentum=0.1),
            nn.LeakyReLU(0.2),
            nn.Linear(config['n_hidden']*2, config['n_hidden']*2),
            nn.BatchNorm1d(config['n_hidden']*2, eps=1e-05, momentum=0.1),
            nn.LeakyReLU(0.2),
            nn.Linear(config['n_hidden']*2, 1),
        )
        self.discriminator.apply(self.init_weights)
        
           
        self.optimizer_AE = optim.SGD(list(self.context_encoder.parameters())
                                      +list(self.post_net.parameters())
                                      +list(self.post_generator.parameters())
                                      +list(self.decoder.parameters()),lr=config['lr_ae'])
        self.optimizer_G = optim.RMSprop(list(self.post_net.parameters())
                                      +list(self.post_generator.parameters())
                                      +list(self.prior_net.parameters())
                                      +list(self.prior_generator.parameters()), lr=config['lr_gan_g'])
        self.optimizer_D = optim.RMSprop(self.discriminator.parameters(), lr=config['lr_gan_d'])
        
        self.lr_scheduler_AE = optim.lr_scheduler.StepLR(self.optimizer_AE, step_size = 10, gamma=0.6)
        
        self.criterion_ce = nn.CrossEntropyLoss()
        
    def init_weights(self, m):
        if isinstance(m, nn.Linear):        
            m.weight.data.uniform_(-0.02, 0.02)
            m.bias.data.fill_(0)
            
    def sample_code_post(self, x, c):
        e, _, _ = self.post_net(torch.cat((x, c),1))
        z = self.post_generator(e)
        return z
   
    def sample_code_prior(self, c):
        e, _, _ = self.prior_net(c)
        z = self.prior_generator(e)
        return z    
    
    def train_AE(self, context, context_lens, utt_lens, floors, response, res_lens):
        self.context_encoder.train()
        self.decoder.train()
        c = self.context_encoder(context, context_lens, utt_lens, floors)
        x,_ = self.utt_encoder(response[:,1:], res_lens-1)      
        z = self.sample_code_post(x, c)
        output = self.decoder(torch.cat((z, c),1), None, response[:,:-1], (res_lens-1))  
        flattened_output = output.view(-1, self.vocab_size) 
        
        dec_target = response[:,1:].contiguous().view(-1)
        mask = dec_target.gt(0) # [(batch_sz*seq_len)]
        masked_target = dec_target.masked_select(mask) # 
        output_mask = mask.unsqueeze(1).expand(mask.size(0), self.vocab_size)# [(batch_sz*seq_len) x n_tokens]
        masked_output = flattened_output.masked_select(output_mask).view(-1, self.vocab_size)
        
        self.optimizer_AE.zero_grad()
        loss = self.criterion_ce(masked_output/self.temp, masked_target)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(list(self.context_encoder.parameters())+list(self.decoder.parameters()), self.clip)
        self.optimizer_AE.step()

        return [('train_loss_AE', loss.item())]
    
    def train_G(self, context, context_lens, utt_lens, floors, response, res_lens): 
        self.context_encoder.eval()
        self.optimizer_G.zero_grad()
        
        for p in self.discriminator.parameters():
            p.requires_grad = False  
        
        c = self.context_encoder(context, context_lens, utt_lens, floors)
        # -----------------posterior samples ---------------------------
        x,_ = self.utt_encoder(response[:,1:], res_lens-1)
        z_post= self.sample_code_post(x.detach(), c.detach())
        errG_post = torch.mean(self.discriminator(torch.cat((z_post, c.detach()),1) ))
        errG_post.backward(minus_one) 
    
        # ----------------- prior samples ---------------------------
        prior_z = self.sample_code_prior(c.detach()) 
        errG_prior = torch.mean(self.discriminator(torch.cat((prior_z, c.detach()),1)))
        errG_prior.backward(one) 
    
        self.optimizer_G.step()
        
        for p in self.discriminator.parameters():
            p.requires_grad = True  
        
        costG = errG_prior - errG_post
        return [('train_loss_G', costG.item())]
    
    def train_D(self, context, context_lens, utt_lens, floors, response, res_lens):
        self.context_encoder.eval()
        self.discriminator.train()
        
        self.optimizer_D.zero_grad()
        
        batch_size=context.size(0)

        c = self.context_encoder(context, context_lens, utt_lens, floors)
        x,_ = self.utt_encoder(response[:,1:], res_lens-1)
        post_z = self.sample_code_post(x, c)
        errD_post = torch.mean(self.discriminator(torch.cat((post_z.detach(), c.detach()),1)))
        errD_post.backward(one)
 
        prior_z = self.sample_code_prior(c) 
        errD_prior = torch.mean(self.discriminator(torch.cat((prior_z.detach(), c.detach()),1)))
        errD_prior.backward(minus_one) 
    
        alpha = gData(torch.rand(batch_size, 1))
        alpha = alpha.expand(prior_z.size())
        interpolates = alpha * prior_z.data + ((1 - alpha) * post_z.data)
        interpolates = Variable(interpolates, requires_grad=True)
        d_input=torch.cat((interpolates, c.detach()),1)
        disc_interpolates = torch.mean(self.discriminator(d_input))
        gradients = torch.autograd.grad(outputs=disc_interpolates, inputs=interpolates,
                               grad_outputs=gData(torch.ones(disc_interpolates.size())),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
        gradient_penalty = ((gradients.contiguous().view(gradients.size(0),-1).norm(2,dim=1)-1)**2).mean()*self.lambda_gp
        gradient_penalty.backward()
    
        self.optimizer_D.step()
        costD = -(errD_prior - errD_post) + gradient_penalty
        return [('train_loss_D', costD.item())]   
    
    def valid(self, context, context_lens, utt_lens, floors, response, res_lens):
        self.context_encoder.eval()      
        self.discriminator.eval()
        self.decoder.eval()
        
        c = self.context_encoder(context, context_lens, utt_lens, floors)
        x,_ = self.utt_encoder(response[:,1:], res_lens-1)
        post_z = self.sample_code_post(x, c)
        prior_z = self.sample_code_prior(c)
        errD_post = torch.mean(self.discriminator(torch.cat((post_z, c),1)))
        errD_prior = torch.mean(self.discriminator(torch.cat((prior_z, c),1)))
        costD = -(errD_prior - errD_post)
        costG = -costD 
        
        dec_target = response[:,1:].contiguous().view(-1)
        mask = dec_target.gt(0) # [(batch_sz*seq_len)]
        masked_target = dec_target.masked_select(mask) 
        output_mask = mask.unsqueeze(1).expand(mask.size(0), self.vocab_size)
        output = self.decoder(torch.cat((post_z, c),1), None, response[:,:-1], (res_lens-1)) 
        flattened_output = output.view(-1, self.vocab_size) 
        masked_output = flattened_output.masked_select(output_mask).view(-1, self.vocab_size)
        lossAE = self.criterion_ce(masked_output/self.temp, masked_target)
        return [('valid_loss_AE', lossAE.item()),('valid_loss_G', costG.item()), ('valid_loss_D', costD.item())]
        
    def sample(self, context, context_lens, utt_lens, floors, repeat, SOS_tok, EOS_tok):    
        self.prior_net.eval()
        self.prior_generator.eval()
        self.context_encoder.eval()
        self.decoder.eval()
        
        c = self.context_encoder(context, context_lens, utt_lens, floors)
#         c_repeated = c.expand(repeat, -1)
        prior_z = self.sample_code_prior(c)
        sample_words, sample_lens= self.decoder.sampling(torch.cat((prior_z, c),1), 
                                                         None, self.maxlen, SOS_tok, EOS_tok, "greedy") 
        return sample_words, sample_lens 
      
    def adjust_lr(self):
        self.lr_scheduler_AE.step()

In [19]:
corpus = getattr(data, 'SWDA'+'Corpus')('../datasets/SWDA/', wordvec_path='../datasets/'+'glove.twitter.27B.200d.txt', wordvec_dim=config['emb_size'])
dials = corpus.get_dialogs()
metas = corpus.get_metas()
vocab = corpus.ivocab
ivocab = corpus.vocab
n_tokens = len(ivocab)
train_dial, valid_dial, test_dial = dials.get("train"), dials.get("valid"), dials.get("test")
train_meta, valid_meta, test_meta = metas.get("train"), metas.get("valid"), metas.get("test")
train_loader = getattr(data, 'SWDA'+'DataLoader')("Train", train_dial, train_meta, config['maxlen'])
valid_loader = getattr(data, 'SWDA'+'DataLoader')("Valid", valid_dial, valid_meta, config['maxlen'])
test_loader = getattr(data, 'SWDA'+'DataLoader')("Test", test_dial, test_meta, config['maxlen'])


Max utt len 96, mean utt len 14.69
Max utt len 75, mean utt len 15.06
Max utt len 74, mean utt len 15.39
Load corpus with train size 3, valid size 3, test size 3 raw vocab size 24497 vocab size 10000 at cut_off 4 OOV rate 0.008035
<d> index 143
<sil> index -1
67 topics in train data
['statement-non-opinion', 'acknowledge_(backchannel)', 'statement-opinion', 'abandoned_or_turn-exit/uninterpretable', 'yes-no-question', 'agree/accept', 'appreciation', 'wh-question', 'backchannel_in_question_form', 'yes_answers', 'conventional-closing', 'response_acknowledgement', 'open-question', 'no_answers', 'affirmative_non-yes_answers', 'declarative_yes-no-question', 'summarize/reformulate', 'other', 'action-directive', 'rhetorical-questions', 'conventional-opening', 'collaborative_completion', 'signal-non-understanding', 'or-clause', 'hold_before_answer/agreement', 'quotation', 'negative_non-no_answers', 'self-talk', 'apology', 'dispreferred_answers', 'offers,_options_commits', 'other_answers', 'reje

In [6]:
corpus = getattr(data, 'DailyDial'+'Corpus')('../datasets/DailyDial/', wordvec_path='../datasets/'+'glove.twitter.27B.200d.txt', wordvec_dim=config['emb_size'])
dials = corpus.get_dialogs()
metas = corpus.get_metas()
vocab = corpus.ivocab
ivocab = corpus.vocab
n_tokens = len(ivocab)
train_dial, valid_dial, test_dial = dials.get("train"), dials.get("valid"), dials.get("test")
train_meta, valid_meta, test_meta = metas.get("train"), metas.get("valid"), metas.get("test")
train_loader = getattr(data, 'DailyDial'+'DataLoader')("Train", train_dial, train_meta, config['maxlen'])
valid_loader = getattr(data, 'DailyDial'+'DataLoader')("Valid", valid_dial, valid_meta, config['maxlen'])
test_loader = getattr(data, 'DailyDial'+'DataLoader')("Test", test_dial, test_meta, config['maxlen'])


Max utt len 296, mean utt len 16.48
Max utt len 174, mean utt len 16.37
Max utt len 214, mean utt len 16.68
Load corpus with train size 2, valid size 2, test size 2 raw vocab size 17716 vocab size 10000 at cut_off 2 OOV rate 0.006757
<d> index 21
<sil> index -1
word2vec cannot cover 0.032194 vocab
Done loading corpus
Max len 36 and min len 3 and avg len 8.840439
Max len 32 and min len 3 and avg len 9.069000
Max len 27 and min len 3 and avg len 8.740000


In [7]:
# 与dialog_doublegan 采用相同的数据集大小
def valid_small(valid_loader):
    model.eval()
    total_loss = 0.0
    total_valid_batch = 0
    valid_count = 0
    with torch.no_grad():
        while True:
            batch = valid_loader.next_batch()
            if batch is None or total_valid_batch >= 1500: # end of epoch
                break
            total_valid_batch += 20
            valid_count += 1
            context, context_lens, utt_lens, floors,_,_,_,response,res_lens,_ = batch
            context, utt_lens = context[:,:,1:], utt_lens-1 # remove the sos token in the context and reduce the context length
            context, context_lens, utt_lens, floors, response, res_lens\
                    = gVar(context), gVar(context_lens), gVar(utt_lens), gData(floors), gVar(response), gVar(res_lens)
            target, outputs = model(context, context_lens, utt_lens, floors, response, res_lens)
            loss_batch = criterion(outputs, target)
            total_loss += float(loss_batch.item())
        return total_loss / valid_count    
    
def sample(context, context_lens, utt_lens, floors, repeat, SOS_tok, EOS_tok):    
    model.eval()
    c = model.context_encoder(context, context_lens, utt_lens, floors)
#     c_repeated = c.expand(repeat, -1)
    sample_words, sample_lens= model.decoder.sampling(c, None, config['maxlen'], SOS_tok, EOS_tok, "greedy")
    return sample_words, sample_lens 

def evaluate(model, metrics, test_loader, vocab, ivocab, f_eval, repeat):
    recall_bleus, prec_bleus, bows_extrema, bows_avg, bows_greedy, intra_dist1s, intra_dist2s, avg_lens, inter_dist1s, inter_dist2s\
        = [], [], [], [], [], [], [], [], [], []
    bleu1_4s = []
    local_t = 0
    test_loader.epoch_init(1, config['diaglen'], 1, shuffle=False)
    valid_count = 0
    begin_time = time()
    all_generated_sentences = []
    all_generated_lens = []
    while True:
        batch = test_loader.next_batch()
        if batch is None:
#         if batch is None or valid_count >= 400:
            break
        valid_count += 1
        local_t += 1 
        context, context_lens, utt_lens, floors,_,_,_,response,res_lens,_ = batch   
        context, utt_lens = context[:,:,1:], utt_lens-1 # remove the sos token in the context and reduce the context length
        f_eval.write("Batch %d \n" % (local_t))# print the context
        start = np.maximum(0, context_lens[0]-5)
        for t_id in range(start, context.shape[1], 1):
            context_str = indexes2sent(context[0, t_id], vocab, vocab["</s>"], 0)
            f_eval.write("Context %d-%d: %s\n" % (t_id, floors[0, t_id], context_str))
        # print the true outputs    
        ref_str, _ = indexes2sent(response[0], vocab, vocab["</s>"], vocab["<s>"])
        ref_tokens = ref_str.split(' ')
        f_eval.write("Target >> %s\n" % (ref_str.replace(" ' ", "'")))
        context, context_lens, utt_lens, floors = gVar(context), gVar(context_lens), gVar(utt_lens), gData(floors)
        sample_words, sample_lens = model.sample(context, context_lens, utt_lens, floors, repeat, vocab["<s>"], vocab["</s>"])
        # 存储所有生成的回复，用来计算div
        all_generated_sentences.append(sample_words[0].tolist())
        all_generated_lens.append(sample_lens[0].tolist())
        # nparray: [repeat x seq_len]
        pred_sents, _ = indexes2sent(sample_words, vocab, vocab["</s>"], 0)
        if valid_count % 300 == 0:
            print('true response: ', ref_str)
            print('generate response: ', pred_sents[0])
        pred_tokens = [sent.split(' ') for sent in pred_sents]
        for r_id, pred_sent in enumerate(pred_sents):
            f_eval.write("Generate >> %s\n" % (pred_sent.replace(" ' ", "'")))
        max_bleu, avg_bleu = metrics.sim_bleu(pred_tokens, ref_tokens)
        recall_bleus.append(max_bleu)
        prec_bleus.append(avg_bleu)
        bleu1_4s.append(metrics.sim_bleu1_4(pred_tokens[0], ref_tokens))
        bow_extrema, bow_avg, bow_greedy = metrics.sim_bow(sample_words, sample_lens, response[:,1:], res_lens-2)
        bows_extrema.append(bow_extrema)
        bows_avg.append(bow_avg)
        bows_greedy.append(bow_greedy)
#         intra_dist1, intra_dist2, inter_dist1, inter_dist2 = metrics.div_distinct(sample_words, sample_lens-1)
#         intra_dist1s.append(intra_dist1)
#         intra_dist2s.append(intra_dist2)
        avg_lens.append(np.mean(sample_lens))
#         inter_dist1s.append(inter_dist1)
#         inter_dist2s.append(inter_dist2)
        f_eval.write("\n")
    recall_bleu = float(np.mean(recall_bleus))
    prec_bleu = float(np.mean(prec_bleus))
    f1 = 2*(prec_bleu*recall_bleu) / (prec_bleu+recall_bleu+10e-12)
    bleu1_4 = np.mean(bleu1_4s, 0)
    bow_extrema = float(np.mean(bows_extrema))
    bow_avg = float(np.mean(bows_avg))
    bow_greedy=float(np.mean(bows_greedy))
#     intra_dist1=float(np.mean(intra_dist1s))
#     intra_dist2=float(np.mean(intra_dist2s))
    avg_len=float(np.mean(avg_lens))
    all_generated_sentences = np.array(all_generated_sentences)
    all_generated_lens = np.array(all_generated_lens)
#     print(all_generated_sentences[:5])
#     print(all_generated_lens[:5])
    intra_dist1, intra_dist2, inter_dist1, inter_dist2 = metrics.div_distinct(all_generated_sentences, all_generated_lens)
#     inter_dist1=float(np.mean(inter_dist1s))
#     inter_dist2=float(np.mean(inter_dist2s))
#     report = "Avg recall BLEU %f, bow_extrema %f, bow_avg %f, bow_greedy %f, inter_dist1 %f, inter_dist2 %f avg_len %f" \
#     % (recall_bleu, bow_extrema, bow_avg, bow_greedy, inter_dist1, inter_dist2, avg_len)
    report = "BLEU1 %f, BLEU2 %f, BLEU3 %f, BLEU4 %f, inter_dist1 %f, inter_dist2 %f avg_len %f" % (bleu1_4[0], bleu1_4[1], bleu1_4[2], bleu1_4[3], inter_dist1, inter_dist2, avg_len)
    f_eval.write(report + "\n")
    print(report)
    print(' time: %.1f s'%(time()-begin_time))
    print("Done testing")
    return recall_bleu, bow_extrema, bow_avg, bow_greedy, inter_dist1, inter_dist2, avg_len

In [22]:
metrics=Metrics(corpus.word2vec)
model = DialogWAE(config, n_tokens)
if corpus.word2vec is not None:
    print("Loaded word2vec")
    model.embedder.weight.data.copy_(torch.from_numpy(corpus.word2vec))
    model.embedder.weight.data[0].fill_(0)
model.to(DEVICE)
model.zero_grad()
print_every = 50
best_state = None
max_metric = 0
for epoch in range(100):
    print('Epoch: ', epoch+1)
    train_loader.epoch_init(32, config['diaglen'], 1, shuffle=True)
    n_iters=train_loader.num_batch
    loss_ae = 0.0
    loss_g = 0.0
    loss_d = 0.0
    epoch_begin = time()
    batch_count = 0
    batch_begin_time = time()
    total_train_batch = 0 # 记录训练的样本数量
    total_valid_batch = 0 # 记录测试的样本数量
    while True:
        model.train()
        loss_records=[]
        batch = train_loader.next_batch()
        total_train_batch += 32
        if batch is None:
#         if batch is None or total_train_batch >= 1000: # end of epoch
            break
        context, context_lens, utt_lens, floors,_,_,_,response,res_lens,_ = batch
        context, utt_lens = context[:,:,1:], utt_lens-1 # remove the sos token in the context and reduce the context length
        context, context_lens, utt_lens, floors, response, res_lens\
                = gVar(context), gVar(context_lens), gVar(utt_lens), gData(floors), gVar(response), gVar(res_lens)
        loss_1 = model.train_AE(context, context_lens, utt_lens, floors, response, res_lens)
        loss_ae += float(loss_1[0][1])
        loss_2 = model.train_G(context, context_lens, utt_lens, floors, response, res_lens)
        loss_g += float(loss_2[0][1])
        
        for i in range(config['n_iters_d']):# train discriminator/critic
            loss_3 = model.train_D(context, context_lens, utt_lens, floors, response, res_lens)  
            if i==0:
                loss_d += float(loss_3[0][1])
            if i==config['n_iters_d']-1:
                break
            batch = train_loader.next_batch()
            if batch is None: # end of epoch
                break
            context, context_lens, utt_lens, floors,_,_,_,response,res_lens,_ = batch
            context, utt_lens = context[:,:,1:], utt_lens-1 # remove the sos token in the context and reduce the context length
            context, context_lens, utt_lens, floors, response, res_lens\
                = gVar(context), gVar(context_lens), gVar(utt_lens), gData(floors), gVar(response), gVar(res_lens)                      
        
        batch_count += 1
        if batch_count % print_every == 0:
            print_flush('[%d %d] loss_ae: %.6f loss_g: %.6f loss_d: %.6f time: %.1f s' %
                  (epoch + 1, batch_count, np.exp(loss_ae / print_every), loss_g / print_every,
                    loss_d / print_every, time() - batch_begin_time))
            loss_ae = 0.0
            loss_g = 0.0
            loss_d = 0.0
            batch_begin_time = time()
    print_flush("Evaluating....")
#     valid_loader.epoch_init(20, config['diaglen'], 1, shuffle=False)
#     loss_valid = valid_small(valid_loader)
#     valid_result.append(F1)
#     print_flush('*'*60)
#     print_flush('[epoch %d]. loss: %.6f time: %.1f s'%(epoch+1, np.exp(loss_valid), time()-epoch_begin))
#     print_flush('*'*60)
#     print_flush("testing....")
#     test_loader.epoch_init(1, config['diaglen'], 1, shuffle=False)
#     loss_valid = valid(test_loader)
#     print_flush('*'*60)
#     print_flush('[epoch %d]. loss: %.6f time: %.1f s'%(epoch+1, np.exp(loss_valid), time()-epoch_begin))
#     print_flush('*'*60)
    if (epoch+1) > 20:
        f_eval = open("../result/{}/{}/epoch{}.txt".format('DialogWAE', 'SWDA', epoch), "w")
        recall_bleu, bow_extrema, bow_avg, bow_greedy, inter_dist1, inter_dist2, avg_len\
         =evaluate(model, metrics, test_loader, vocab, ivocab, f_eval, repeat=1)
    epoch_begin = time()
#     if F1 > max_metric:
#         best_state = model.state_dict()
#         max_metric = F1
#         print_flush("save model...")
#         torch.save(best_state, '../datasets/models/baseline_LSTM.pth')
#     epoch_begin = time()
#     if training_termination(valid_result):
#         print_flush("early stop at [%d] epoch!" % (epoch+1))
#         break
    model.adjust_lr()

Loaded word2vec
Epoch:  1
Train begins with 6398 batches with 12 left over samples
[1 50] loss_ae: 772.952671 loss_g: 0.003260 loss_d: 9.441190 time: 7.1 s
[1 100] loss_ae: 281.586226 loss_g: 0.007951 loss_d: 9.201698 time: 6.9 s
[1 150] loss_ae: 176.866460 loss_g: 0.021754 loss_d: 9.092923 time: 7.0 s
[1 200] loss_ae: 146.034011 loss_g: 0.029456 loss_d: 8.941715 time: 7.0 s
[1 250] loss_ae: 127.633686 loss_g: 0.038266 loss_d: 8.804258 time: 7.0 s
[1 300] loss_ae: 117.296023 loss_g: 0.084724 loss_d: 8.665235 time: 7.0 s
[1 350] loss_ae: 103.915196 loss_g: 0.089025 loss_d: 8.589442 time: 7.0 s
[1 400] loss_ae: 91.235519 loss_g: 0.078870 loss_d: 8.492210 time: 7.0 s
[1 450] loss_ae: 86.707661 loss_g: 0.069349 loss_d: 8.387099 time: 7.0 s
[1 500] loss_ae: 84.711306 loss_g: 0.085879 loss_d: 8.203546 time: 7.0 s
[1 550] loss_ae: 84.355394 loss_g: 0.084642 loss_d: 8.040713 time: 7.0 s
[1 600] loss_ae: 76.815586 loss_g: 0.099949 loss_d: 7.835270 time: 7.0 s
[1 650] loss_ae: 75.378659 loss_g: 

[5 350] loss_ae: 37.859651 loss_g: 6.218454 loss_d: -3.364293 time: 6.8 s
[5 400] loss_ae: 36.183868 loss_g: 6.302505 loss_d: -3.560900 time: 6.7 s
[5 450] loss_ae: 33.951994 loss_g: 6.429477 loss_d: -3.783187 time: 6.7 s
[5 500] loss_ae: 37.703691 loss_g: 6.518563 loss_d: -3.748051 time: 6.7 s
[5 550] loss_ae: 35.514817 loss_g: 6.578220 loss_d: -3.797254 time: 6.7 s
[5 600] loss_ae: 36.086232 loss_g: 6.717169 loss_d: -3.815160 time: 6.8 s
[5 650] loss_ae: 38.782208 loss_g: 7.205209 loss_d: -4.488389 time: 6.8 s
[5 700] loss_ae: 37.765013 loss_g: 7.306391 loss_d: -4.605716 time: 6.8 s
[5 750] loss_ae: 33.293509 loss_g: 6.983238 loss_d: -4.192238 time: 6.8 s
[5 800] loss_ae: 33.331662 loss_g: 7.339334 loss_d: -4.687312 time: 6.7 s
[5 850] loss_ae: 35.962166 loss_g: 7.493131 loss_d: -4.819886 time: 6.8 s
[5 900] loss_ae: 35.991774 loss_g: 7.853860 loss_d: -5.241279 time: 6.7 s
[5 950] loss_ae: 35.134504 loss_g: 8.025310 loss_d: -5.515351 time: 6.7 s
[5 1000] loss_ae: 32.883906 loss_g: 8.

[9 550] loss_ae: 27.186585 loss_g: 30.330113 loss_d: -28.816940 time: 9.7 s
[9 600] loss_ae: 28.296203 loss_g: 31.404423 loss_d: -29.884428 time: 9.6 s
[9 650] loss_ae: 28.992795 loss_g: 32.532437 loss_d: -31.093220 time: 9.7 s
[9 700] loss_ae: 25.947943 loss_g: 31.456332 loss_d: -30.074452 time: 9.8 s
[9 750] loss_ae: 26.714921 loss_g: 31.091417 loss_d: -29.636735 time: 9.7 s
[9 800] loss_ae: 24.871557 loss_g: 32.259931 loss_d: -30.757568 time: 9.7 s
[9 850] loss_ae: 28.846435 loss_g: 32.680481 loss_d: -31.104310 time: 9.8 s
[9 900] loss_ae: 27.489551 loss_g: 33.479888 loss_d: -31.993742 time: 9.6 s
[9 950] loss_ae: 25.767852 loss_g: 33.775568 loss_d: -32.205791 time: 9.6 s
[9 1000] loss_ae: 25.624436 loss_g: 32.484541 loss_d: -30.946149 time: 9.6 s
[9 1050] loss_ae: 26.808944 loss_g: 31.962697 loss_d: -30.184419 time: 9.7 s
[9 1100] loss_ae: 27.484942 loss_g: 32.816767 loss_d: -31.329340 time: 9.7 s
[9 1150] loss_ae: 29.139065 loss_g: 34.913741 loss_d: -33.506772 time: 9.7 s
[9 1200]

[13 650] loss_ae: 20.147793 loss_g: 39.559189 loss_d: -35.983067 time: 9.8 s
[13 700] loss_ae: 21.377983 loss_g: 38.643849 loss_d: -34.895344 time: 9.7 s
[13 750] loss_ae: 22.262758 loss_g: 34.967984 loss_d: -31.698883 time: 9.7 s
[13 800] loss_ae: 20.861565 loss_g: 37.047480 loss_d: -33.475615 time: 9.7 s
[13 850] loss_ae: 22.427043 loss_g: 39.427452 loss_d: -34.851948 time: 9.8 s
[13 900] loss_ae: 24.014749 loss_g: 34.902353 loss_d: -30.447578 time: 9.7 s
[13 950] loss_ae: 21.324488 loss_g: 33.862068 loss_d: -29.388565 time: 9.7 s
[13 1000] loss_ae: 20.321437 loss_g: 34.198361 loss_d: -29.761123 time: 9.5 s
[13 1050] loss_ae: 21.351447 loss_g: 34.231571 loss_d: -31.460623 time: 9.8 s
[13 1100] loss_ae: 23.056334 loss_g: 37.965926 loss_d: -34.086987 time: 9.6 s
[13 1150] loss_ae: 22.303441 loss_g: 32.989465 loss_d: -28.432210 time: 9.7 s
[13 1200] loss_ae: 21.254090 loss_g: 33.655690 loss_d: -29.491164 time: 9.6 s
[13 1250] loss_ae: 22.074044 loss_g: 35.200301 loss_d: -30.936599 time:

[17 950] loss_ae: 18.575243 loss_g: 37.636724 loss_d: -32.320322 time: 8.4 s
[17 1000] loss_ae: 17.308367 loss_g: 33.547353 loss_d: -29.296290 time: 8.1 s
[17 1050] loss_ae: 18.388342 loss_g: 36.622879 loss_d: -33.363054 time: 8.4 s
[17 1100] loss_ae: 18.355875 loss_g: 35.853897 loss_d: -30.095099 time: 8.4 s
[17 1150] loss_ae: 18.858322 loss_g: 34.428349 loss_d: -30.245642 time: 8.3 s
[17 1200] loss_ae: 21.042025 loss_g: 36.157712 loss_d: -33.849894 time: 8.2 s
[17 1250] loss_ae: 17.914418 loss_g: 35.508517 loss_d: -31.905805 time: 8.3 s
Evaluating....
Epoch:  18
Train begins with 6398 batches with 12 left over samples
[18 50] loss_ae: 20.126635 loss_g: 35.524317 loss_d: -31.672435 time: 15.5 s
[18 100] loss_ae: 20.588881 loss_g: 34.070028 loss_d: -29.479850 time: 20.8 s
[18 150] loss_ae: 18.720538 loss_g: 34.022009 loss_d: -31.305369 time: 21.0 s
[18 200] loss_ae: 19.336663 loss_g: 35.493099 loss_d: -32.130773 time: 20.7 s
[18 250] loss_ae: 19.965893 loss_g: 37.532155 loss_d: -32.129

...

true response:  yeah </s>
generate response:  yeah </s>
true response:  <unk> of people ' s heart strings and making them cry and feel like they ' re doing somebody good by giving them </s>
generate response:  you know with <unk> you can have a <unk> you know you have to have a <unk> you know </s>
true response:  something like that </s>
generate response:  yeah what ' s interesting is that you ' ve got to have a lot of <unk> and stuff like that and i ' m not sure </s>
true response:  yeah that ' s fun </s>
generate response:  uh - huh </s>
true response:  oh </s>
generate response:  oh yeah </s>
true response:  lay on those beaches over there </s>
generate response:  and to get they have to have you know a <unk> or a <unk> or something like that you know i mean i ' ve heard </s>
true response:  uh - huh </s>
generate response:  um - hum </s>
true response:  they before we got her everybody said oh they ' re just like a cat you know </s>
generate response:  or they are not sure how lon

[95 550] loss_ae: 11.922449 loss_g: 44.677827 loss_d: -36.697819 time: 8.2 s
[95 600] loss_ae: 12.879628 loss_g: 51.886022 loss_d: -39.424097 time: 8.2 s
[95 650] loss_ae: 13.483893 loss_g: 44.629009 loss_d: -32.939882 time: 8.3 s
[95 700] loss_ae: 14.304925 loss_g: 42.675052 loss_d: -28.517915 time: 8.3 s
[95 750] loss_ae: 13.490675 loss_g: 36.880632 loss_d: -27.012731 time: 8.3 s
[95 800] loss_ae: 13.237393 loss_g: 38.154242 loss_d: -34.201610 time: 8.3 s
[95 850] loss_ae: 12.780832 loss_g: 43.941451 loss_d: -39.361835 time: 8.3 s
[95 900] loss_ae: 13.804437 loss_g: 41.996913 loss_d: -33.601454 time: 8.2 s
[95 950] loss_ae: 13.599225 loss_g: 52.486835 loss_d: -40.792955 time: 8.3 s
[95 1000] loss_ae: 14.553553 loss_g: 48.572036 loss_d: -38.068780 time: 8.2 s
[95 1050] loss_ae: 14.294220 loss_g: 47.348842 loss_d: -35.610943 time: 8.4 s
[95 1100] loss_ae: 14.020874 loss_g: 47.286647 loss_d: -35.619220 time: 8.1 s
[95 1150] loss_ae: 14.100567 loss_g: 45.994283 loss_d: -35.708619 time: 8

true response:  yeah well see that ' s the reason that we couldn ' t make really make them at first we were going to get a pick up truck </s>
generate response:  um - hum </s>
true response:  you ' d think i ' d have a lot time but </s>
generate response:  you have to have a you have to have a you have a you have a you have a </s>
true response:  and </s>
generate response:  they are they are they are they are they are they are they are you know a lot of times they are they are they are they are </s>
true response:  yeah i ' ve got a well my roses are on the west side of the house i asked my neighbors what they wanted to see outside their </s>
generate response:  yeah you can do you can ' t get anything like that </s>
true response:  um - hum </s>
generate response:  um - hum </s>
BLEU1 0.417832, BLEU2 0.342708, BLEU3 0.290364, BLEU4 0.233675, inter_dist1 0.006277, inter_dist2 0.039266 avg_len 16.421273
 time: 145.5 s
Done testing
Epoch:  97
Train begins with 6398 batches with 12 left 

true response:  something like that </s>
generate response:  they are they are they are they are they are they are they are they are they are they are they are they are they are they are they are
true response:  yeah that ' s fun </s>
generate response:  and you can always get you can get them out of there and you can ' t get them in the house and </s>
true response:  oh </s>
generate response:  if they do not do they do they do they do they do they do they do it outside of the house and they do that they </s>
true response:  lay on those beaches over there </s>
generate response:  and have been having to have a lot of <unk> and i have a lot of <unk> and i have a lot of <unk> and i have </s>
true response:  uh - huh </s>
generate response:  uh - huh </s>
true response:  they before we got her everybody said oh they ' re just like a cat you know </s>
generate response:  they are they are they are they are they are they are they are they are they are they are they are they are they are t

[100 450] loss_ae: 15.087776 loss_g: 45.159494 loss_d: -34.218481 time: 8.5 s
[100 500] loss_ae: 14.233923 loss_g: 43.753378 loss_d: -45.501710 time: 8.4 s
[100 550] loss_ae: 13.344636 loss_g: 51.137384 loss_d: -37.833727 time: 8.4 s
[100 600] loss_ae: 14.683267 loss_g: 42.967670 loss_d: -32.429834 time: 8.3 s
[100 650] loss_ae: 13.145862 loss_g: 40.106173 loss_d: -34.415643 time: 8.4 s
[100 700] loss_ae: 13.071185 loss_g: 41.093087 loss_d: -31.705087 time: 8.2 s
[100 750] loss_ae: 12.726765 loss_g: 46.139163 loss_d: -36.457487 time: 8.4 s
[100 800] loss_ae: 13.525625 loss_g: 39.800800 loss_d: -26.833157 time: 8.2 s
[100 850] loss_ae: 13.825599 loss_g: 43.555780 loss_d: -27.418980 time: 8.5 s
[100 900] loss_ae: 14.398101 loss_g: 42.999491 loss_d: -26.528606 time: 8.2 s
[100 950] loss_ae: 13.477579 loss_g: 37.928825 loss_d: -31.887104 time: 8.4 s
[100 1000] loss_ae: 13.719309 loss_g: 34.090738 loss_d: -34.493382 time: 8.3 s
[100 1050] loss_ae: 12.965043 loss_g: 42.403228 loss_d: -36.693